In [169]:
import json
import requests as rs
import requests_cache 

requests_cache.install_cache(cache_name='discogs_cache', backend='sqlite', expire_after=86400)

release_id = "1272706"
min_average = 4.7

In [170]:
url = "https://api.discogs.com/labels/{}/releases?page=1&per_page=25".format(release_id)
response = rs.get(url)
print(response)
response_dict = json.loads(response.text)
print(json.dumps(response_dict, indent=4, sort_keys=True))

<CachedResponse [200]: created: 2023-12-30 17:35:24 CET, expires: 2023-12-31 17:35:24 CET (fresh), size: 3.93 KiB, request: GET https://api.discogs.com/labels/1272706/releases?page=1&per_page=25>
{
    "pagination": {
        "items": 18,
        "page": 1,
        "pages": 1,
        "per_page": 25,
        "urls": {}
    },
    "releases": [
        {
            "artist": "ANN\u0112",
            "catno": "LIP007",
            "format": "6xFile, FLAC",
            "id": 26146235,
            "resource_url": "https://api.discogs.com/releases/26146235",
            "status": "Accepted",
            "thumb": "",
            "title": "Trinity",
            "year": 2023
        },
        {
            "artist": "ANN\u0112",
            "catno": "LIP007",
            "format": "6xFile, FLAC",
            "id": 26146235,
            "resource_url": "https://api.discogs.com/releases/26146235",
            "status": "Accepted",
            "thumb": "",
            "title": "Trinity",
      

In [171]:
response_rating_list = []
for release in response_dict["releases"]:
    # Get community release rating
    resource_url = release["resource_url"]
    rating_url = resource_url + "/rating"
    # print(rating_url)
    response_rating = rs.get(rating_url)
    # print("Used Cache: {0}".format(response_rating.from_cache))
    response_rating_dict = json.loads(response_rating.text)
    # print(response_rating_dict)
    # print(json.dumps(response_rating_dict, indent=4, sort_keys=True))
    response_rating_list.append(response_rating_dict) 

for item in response_rating_list:
   print(item) 

{'release_id': 26146235, 'rating': {'count': 0, 'average': 0.0}}
{'release_id': 26146235, 'rating': {'count': 0, 'average': 0.0}}
{'release_id': 14704507, 'rating': {'count': 14, 'average': 4.79}}
{'release_id': 14704507, 'rating': {'count': 14, 'average': 4.79}}
{'release_id': 10502778, 'rating': {'count': 13, 'average': 4.15}}
{'release_id': 10502778, 'rating': {'count': 13, 'average': 4.15}}
{'release_id': 10502778, 'rating': {'count': 13, 'average': 4.15}}
{'release_id': 12402975, 'rating': {'count': 10, 'average': 4.3}}
{'release_id': 14704507, 'rating': {'count': 14, 'average': 4.79}}
{'release_id': 17282449, 'rating': {'count': 20, 'average': 4.75}}
{'release_id': 23819831, 'rating': {'count': 17, 'average': 4.88}}
{'release_id': 23824595, 'rating': {'count': 1, 'average': 5.0}}
{'release_id': 24753029, 'rating': {'count': 13, 'average': 4.77}}
{'release_id': 26146235, 'rating': {'count': 0, 'average': 0.0}}
{'release_id': 26041843, 'rating': {'count': 16, 'average': 4.69}}
{'re

In [172]:
# Filter releases with average >= min_average
filtered_dict_list = [item for item in response_rating_list if item["rating"]["average"] >= min_average]

for item in filtered_dict_list:
    print(item)

{'release_id': 14704507, 'rating': {'count': 14, 'average': 4.79}}
{'release_id': 14704507, 'rating': {'count': 14, 'average': 4.79}}
{'release_id': 14704507, 'rating': {'count': 14, 'average': 4.79}}
{'release_id': 17282449, 'rating': {'count': 20, 'average': 4.75}}
{'release_id': 23819831, 'rating': {'count': 17, 'average': 4.88}}
{'release_id': 23824595, 'rating': {'count': 1, 'average': 5.0}}
{'release_id': 24753029, 'rating': {'count': 13, 'average': 4.77}}


In [173]:
# Create a dictionary mapping release_id to rating_info
rating_info_dict = {item["release_id"]: item["rating"] for item in filtered_dict_list}

# Extract information from response_dict only for matching release_ids
matching_releases = [
    {
        "release_info": release_info,
        "rating_info": rating_info_dict.get(release_info["id"])
    }
    for release_info in response_dict["releases"] if release_info["id"] in rating_info_dict
]

# Print the matching releases
for release in matching_releases:
    print("Release Info:")
    print(json.dumps(release["release_info"], indent=4, sort_keys=True))
    print("Rating Info:")
    print(json.dumps(release["rating_info"], indent=4, sort_keys=True))
    print()

Release Info:
{
    "artist": "Ferdinger",
    "catno": "LIP003",
    "format": "12\", EP",
    "id": 14704507,
    "resource_url": "https://api.discogs.com/releases/14704507",
    "status": "Accepted",
    "thumb": "",
    "title": "Euphoria",
    "year": 2020
}
Rating Info:
{
    "average": 4.79,
    "count": 14
}

Release Info:
{
    "artist": "Ferdinger",
    "catno": "LIP003",
    "format": "12\", EP",
    "id": 14704507,
    "resource_url": "https://api.discogs.com/releases/14704507",
    "status": "Accepted",
    "thumb": "",
    "title": "Euphoria",
    "year": 2020
}
Rating Info:
{
    "average": 4.79,
    "count": 14
}

Release Info:
{
    "artist": "Ferdinger",
    "catno": "LIP003",
    "format": "12\", EP",
    "id": 14704507,
    "resource_url": "https://api.discogs.com/releases/14704507",
    "status": "Accepted",
    "thumb": "",
    "title": "Euphoria",
    "year": 2020
}
Rating Info:
{
    "average": 4.79,
    "count": 14
}

Release Info:
{
    "artist": "Various",
  

In [180]:
# Create specific output with changed URL
output_list = []
for release in matching_releases:
    new_dict = {
        "catno": release["release_info"]["catno"],
        "artist": release["release_info"]["artist"],
        "title": release["release_info"]["title"],
        "year": release["release_info"]["year"],
        "format": release["release_info"]["format"],
        "url": "https://www.discogs.com/es/release/{}".format(release["release_info"]["id"]),
        "rating": release["rating_info"]
    }
    output_list.append(new_dict)

# print(output_list)
print(json.dumps(output_list, indent=4))

[
    {
        "catno": "LIP003",
        "artist": "Ferdinger",
        "title": "Euphoria",
        "year": 2020,
        "format": "12\", EP",
        "url": "https://www.discogs.com/es/release/14704507",
        "rating": {
            "count": 14,
            "average": 4.79
        }
    },
    {
        "catno": "LIP003",
        "artist": "Ferdinger",
        "title": "Euphoria",
        "year": 2020,
        "format": "12\", EP",
        "url": "https://www.discogs.com/es/release/14704507",
        "rating": {
            "count": 14,
            "average": 4.79
        }
    },
    {
        "catno": "LIP003",
        "artist": "Ferdinger",
        "title": "Euphoria",
        "year": 2020,
        "format": "12\", EP",
        "url": "https://www.discogs.com/es/release/14704507",
        "rating": {
            "count": 14,
            "average": 4.79
        }
    },
    {
        "catno": "LIP004",
        "artist": "Various",
        "title": "Spektrum",
        "year": 